In [164]:
import numpy as np
import copy
from func import conflictcalculator, link_maker
from itertools import product, combinations

In [222]:
class game_node():
    def __init__(self, playfield, g = 0, f = 0, h = 0, rc = False ) -> None:
        l = locals()
        [self.__setattr__(name, l[name]) for name in self.__init__.__code__.co_varnames if name != "self"]
    
    def calc_score(self):
        return self.g+self.f-self.h

    def __eq__(self, other):
        o1, o2 = self.calc_score(), other.calc_score()
        if (o1 == o2) and (self.rc == other.rc):
            return True
        return False

    def __lt__(self, other): #is self less than other
        o1, o2 = self.calc_score(), other.calc_score()
        if o1 == o2:
            if self.g == other.g:
                if self.f == other.f:
                    if self.h == other.h:
                        if self.rc == other.rc:
                            return False # objects are equal
                        else:
                            return self.rc
                    else:
                        return self.h > other.h
                else:
                    return self.f < other.f
            else:
                return self.g < other.g
        else:
            return o1 < o2

    def __repr__(self):
        return str(self.playfield) + f" g = {self.g}, f = {self.f}, h = {self.h}, rc = {self.rc}"

In [223]:
def list_replace_copy(list, idx, val):
    nl = copy.copy(list)
    nl[idx] = val
    return nl

In [224]:
size = 4
cc = conflictcalculator(size)
# init_playfield = np.random.randint(0, size, size)
full_set = {i for i in range(size)}
init_playfield = [-1, 1, -1, -1] #for now only support this

In [233]:
fringe = [game_node(init_playfield)] #make self-sorting list
breaker = 0
while len(fringe) > 0:
    breaker += 1
    if breaker > 3:
        break
    op = fringe.pop()
    print(op)
    conflicts = {idx:100 for idx, i in enumerate(op.playfield) if i == -1} # in the future, order by num conflicts per column, consider row conflicts as more important
    print(conflicts)
    new_min_c = conflicts[sorted(conflicts, key = lambda k: conflicts[k])[0]]
    # new_pos = np.random.choice([i for i,k in conflicts.items() if k == new_min_c])
    new_pos = [i for i,k in conflicts.items() if k == new_min_c][0]
    print(new_pos)
    for pot_col in range(size):
        pot_playspace = list_replace_copy(op.playfield, new_pos, pot_col)
        g = sum([True for i in pot_playspace if i != -1]) - 1

        open_columns = [idx for idx,i in enumerate(pot_playspace) if i == -1]
        closed_columns = full_set.difference(set(open_columns))
        h = 0
        for c1, c2 in combinations(closed_columns,2):
            x = cc.r_calc(pot_playspace[c1], c1, size)
            if((pot_playspace[c2] in x[c1]) or (pot_playspace[c2] in x[c2])):
                # print("conflict found", c1,c2)
                h += 1
        
        full_spaces = set(product(full_set, open_columns))
        for full_col_idx in closed_columns:
            x = cc.r_calc(pot_playspace[full_col_idx], full_col_idx, size)
            broken_spaces = set()
            for r, c in full_spaces:
                if ((r in x[full_col_idx]) or (r in x[c])):
                    broken_spaces.add((r,c))
            full_spaces = full_spaces.difference(broken_spaces)
        f = len(full_spaces)
        rc = len(x:=([pot_playspace[i] for i in closed_columns])) != len(np.unique(x))
        print(pot_playspace, g, f, h, "\n")
        fringe.append(game_node(pot_playspace, g, f, h, rc))
        # print(x, "\n")
        #calculate g, f, h
    fringe = sorted(fringe)


[-1, 1, -1, -1] g = 0, f = 0, h = 0, rc = False
{0: 100, 2: 100, 3: 100}
0
[0, 1, -1, -1] 1 2 1 

[1, 1, -1, -1] 1 2 1 

[2, 1, -1, -1] 1 2 1 

[3, 1, -1, -1] 1 1 0 

[2, 1, -1, -1] g = 1, f = 2, h = 1, rc = False
{2: 100, 3: 100}
2
[2, 1, 0, -1] 2 0 3 

[2, 1, 1, -1] 2 0 2 

[2, 1, 2, -1] 2 1 3 

[2, 1, 3, -1] 2 1 1 

[2, 1, 3, -1] g = 2, f = 1, h = 1, rc = False
{3: 100}
3
[2, 1, 3, 0] 3 0 1 

[2, 1, 3, 1] 3 0 2 

[2, 1, 3, 2] 3 0 3 

[2, 1, 3, 3] 3 0 3 



In [234]:
fringe

[[2, 1, 0, -1] g = 2, f = 0, h = 3, rc = False,
 [2, 1, 1, -1] g = 2, f = 0, h = 2, rc = True,
 [2, 1, 2, -1] g = 2, f = 1, h = 3, rc = True,
 [2, 1, 3, 2] g = 3, f = 0, h = 3, rc = True,
 [2, 1, 3, 3] g = 3, f = 0, h = 3, rc = True,
 [2, 1, 3, 1] g = 3, f = 0, h = 2, rc = True,
 [3, 1, -1, -1] g = 1, f = 1, h = 0, rc = False,
 [1, 1, -1, -1] g = 1, f = 2, h = 1, rc = True,
 [0, 1, -1, -1] g = 1, f = 2, h = 1, rc = False,
 [2, 1, 3, 0] g = 3, f = 0, h = 1, rc = False]